*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

# Load the Data and Preview

In [2]:
# Get data by running code at bottom of notebook

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'
users_filename = 'BX-Users.csv'

In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
df_ratings

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0
...,...,...,...
1149775,276704,1563526298,9.0
1149776,276706,0679447156,0.0
1149777,276709,0515107662,10.0
1149778,276721,0590442449,10.0


In [5]:
df_books

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber
...,...,...,...
271374,0440400988,There's a Bat in Bunk Five,Paula Danziger
271375,0525447644,From One to One Hundred,Teri Sloat
271376,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker
271377,0192126040,Republic (World's Classics),Plato


# Prepare Features for Modeling

In [6]:
# https://towardsdatascience.com/prototyping-a-recommender-system-step-by-step-part-1-knn-item-based-collaborative-filtering-637969614ea
# pivot ratings into movie features
users_with_200_ratings = lambda x: len(x) >= 200
books_with_100_ratings = lambda x: len(x) >= 100

user_index = (df_ratings
    .groupby("user")
    .filter(users_with_200_ratings)
    .index)

book_index = (df_ratings
    .groupby("isbn")
    .filter(books_with_100_ratings)
    .index)

feature_index = book_index.intersection(user_index)
features = df_ratings.index.isin(feature_index)

# Prepare the KNN Model

In [35]:
ratings = (df_ratings[features]
    .pivot(index='isbn', columns='user', values='rating')
    .fillna(0))
ratings_matrix = csr_matrix(ratings.values)
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=5, n_jobs=-1)
model = knn.fit(ratings_matrix)

# Define Recommendation Function and Helpers

In [36]:
def isbn_by_title(books):
    def inner(book):
        return books.isbn[df_books.title == book].values[0]
    return inner


def index_by_isbn(ratings):
    def inner(isbn):
        return ratings.index.get_loc(isbn)
    return inner


def index_by_title(books, ratings):
    def inner(book):
        return index_by_isbn(ratings)(isbn_by_title(books)(book))
    return inner
    

def neighbors(X):
    def inner(pos, model, k):
        return [item[pos] for item in model.kneighbors(X, n_neighbors=k)]
    return inner


def isbn_by_index(ratings):
    def inner(idx):
        return ratings.index[idx]
    return inner


def title_by_isbn(books):
    def inner(isbn):
        return books.title[books.isbn == isbn].values[0]
    return inner


In [69]:
def recommender(
    model, 
    k = 5, 
    find_isbn_by_title = isbn_by_title(df_books),
    find_index_by_isbn = index_by_isbn(ratings),
    find_neighbors = neighbors(ratings_matrix),
    find_isbn_by_index = isbn_by_index(ratings),
    find_title_by_isbn = title_by_isbn(df_books)):
    
    def find_index(book):
        isbn = find_isbn_by_title(book)
        return find_index_by_isbn(isbn)
    
    def find_book(index):
        isbn = find_isbn_by_index(index)
        return find_title_by_isbn(isbn)
    
    def recommend(book):
        distances, positions = find_neighbors(find_index(book), model, k)
        neighbors = zip(positions, distances)
        titles = [[find_book(pos), score] for pos, score in neighbors]
        recommendation = [
            book, 
            sorted(titles[1:], key=lambda x: -x[1])
        ]
        return recommendation
    return recommend

get_recommends = recommender(model)

Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [70]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", "The Weight of Water", "The Surgeon", "I Know This Much Is True"]
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉


# Run Once

Load data

In [21]:
import requests
import os
import io
import zipfile

url = "https://cdn.freecodecamp.org/project-data/books/book-crossings.zip"
req = requests.get(url).content
zip = zipfile.ZipFile(io.BytesIO(req))
zip.extractall(".")